### Transfer learning

Install and import packages

In [1]:
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install torch
%pip install torchvision
%pip install helper

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\natha\OneDrive\Skrivbord\Deep learning\Deep Learning Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\natha\OneDrive\Skrivbord\Deep learning\Deep Learning Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


...

You should consider upgrading via the 'c:\Users\natha\OneDrive\Skrivbord\Deep learning\Deep Learning Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\natha\OneDrive\Skrivbord\Deep learning\Deep Learning Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\natha\OneDrive\Skrivbord\Deep learning\Deep Learning Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch as th
from torch.utils.data import DataLoader
import torchvision as tv
from torchvision import transforms
import os

### Load dataset
Maybe normalize dataset

In [3]:
from torch.utils.data import Dataset
from PIL import Image

def open_image(path): # https://jovian.ai/aakashns/transfer-learning-pytorch
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
class MyDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.files = self.initPaths()
        self.classes = self.initClasses()
        self.classesCatDog = self.initClassesCatDog()
        self.transform = transforms.Compose([transforms.Resize(255),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor()]) # Change this so we undertand
    
    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        path = "/data/images/" + self.files[i]
        img = self.transform(open_image(path))
        class_idx = self.classes.index(self.getClass(self.files[i]))
        return img, class_idx
    
    def getClass(self, fileName):
       nameList = fileName.split("_")
       name = ""
       for namePart in nameList:
           if ".jpg" not in namePart:
               name += namePart
       return name
        
    def initClasses(self):
        classList = []
        for fname in self.files:
            theClass = self.getClass(fname)
            if theClass not in classList:
                classList.append(theClass)
        return classList
    
    def getClassCatDog(self, fileName):
       if fileName[0].isupper():
           return "cat"
       else:
           return "dog"
    
    def initClassesCatDog(self):
        classList = []
        for fname in self.files:
            theClass = self.getClass(fname)
            if theClass not in classList:
                classList.append(theClass)
        return ["cat", "dog"]
    
    def initPaths(self):
        files = os.listdir("./data/images/")
        jpgList = []
        for file in files:
            if file.endswith(".jpg"):
                jpgList.append(file)

        return jpgList
    

test = MyDataset()

print(test.classes, test)

['Abyssinian', 'americanbulldog', 'americanpitbullterrier', 'bassethound', 'beagle', 'Bengal', 'Birman', 'Bombay', 'boxer', 'BritishShorthair', 'chihuahua', 'EgyptianMau', 'englishcockerspaniel', 'englishsetter', 'germanshorthaired', 'greatpyrenees', 'havanese', 'japanesechin', 'keeshond', 'leonberger', 'MaineCoon', 'miniaturepinscher', 'newfoundland', 'Persian', 'pomeranian', 'pug', 'Ragdoll', 'RussianBlue', 'saintbernard', 'samoyed', 'scottishterrier', 'shibainu', 'Siamese', 'Sphynx', 'staffordshirebullterrier', 'wheatenterrier', 'yorkshireterrier'] <__main__.MyDataset object at 0x000001F2A8093C70>


In [4]:
from torch.utils.data import random_split

def initDataset():
    dataset = MyDataset()
    
    # Train/Validation/Test split
    train, test = random_split(dataset, [int(0.85*len(dataset))+1, int(0.15*len(dataset))])
    valid, train = random_split(train, [int(0.176*len(train))+1, int((1-0.176)*len(train))]) # Rounding error
    
    # print(len(train), len(valid), len(test))
    # print(len(train)+ len(valid)+ len(test), len(dataset))
    
    train_loader = th.utils.data.DataLoader(train,
                                            batch_size=32,
                                            shuffle=True,
                                            num_workers=4)
    test_loader = th.utils.data.DataLoader(test,
                                            batch_size=32,
                                            shuffle=True,
                                            num_workers=4)
    valid_loader = th.utils.data.DataLoader(valid,
                                            batch_size=32,
                                            shuffle=True,
                                            num_workers=4)
    
    return train_loader , test_loader, valid_loader

train_loader, test_loader, valid_loader = initDataset()

### Show images

In [5]:
valIterator = iter(valid_loader)
trainIterator = iter(train_loader)
testIterator = iter(test_loader)

In [ ]:
def test(iter):
    for image, labels in iter:
        plt.imshow(image[0].permute(1, 2, 0))
        plt.show()
        print(labels[0])
test(valIterator)

### Initialize model

In [ ]:
model = tv.models.resnet18(progress = True)